In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time

<img src="MLP.png" style="width:700px; height:900">

In [ ]:
Ns = [5, 7, 9, 11, 13, 15] # number of neurons in the hidden layer
n = 2 # number of features

# The y_hat function

<img src="y_hat.png" style="width:700px; height:900">

In [ ]:
# define the prediction function
def y_hat(w, b, v, x):
    y_h = np.zeros(len(x))
    wb = np.hstack((w,np.reshape(b, (-1, 1))))
    for i in range(len(x)):
        row = x[i]
        dot_product = np.tanh(np.dot(wb, np.reshape(row, (-1, 1))))  # Perform W.X
        result = np.dot(np.reshape(dot_product, (1, -1)), np.reshape(v, (-1, 1)))  # Perform (W.X).V
        y_h[i] = result
    return y_h

In [ ]:
# define the loss function by defining the MSE then the regularized error
def mse(y_h, y):
    return sum((y_h-y)**2)/len(y)

In [ ]:
# reading the data
columns = ['X1', 'X2', 'Y']
df = pd.read_csv('Dataset.csv', names= columns) 
df.head()

In [ ]:
df['ones'] = np.ones(250)

In [ ]:
x = df.drop(columns= "Y")
x = x.to_numpy()
y = df["Y"].to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25, random_state= 2057785) 

In [ ]:
train_data = np.hstack((x_train, np.reshape(y_train, (-1, 1))))

In [ ]:
rhos = [1e-05, 1e-04, 1e-03, 1e-02]

# The error function

<img src="E.png" style="width:700px; height:900">

In [ ]:
# the error function
def err_fn(W, train_data):
    X_train = train_data[:, 0:3]
    Y_train = train_data[:, 3]
    w = np.reshape(W[0:n*N],(N,n))
    v = np.reshape(W[n*N:n*N+N],(N,1))
    b = np.reshape(W[n*N+N:n*N+2*N],(N,1))
    y_h = y_hat(w, b, v, X_train) # predicted y (yhat)
    e_mse = mse(y_h, Y_train) # mean squared error
    regularization_term = 0.5 * rho * (np.linalg.norm(w) ** 2 + np.linalg.norm(v) ** 2 + np.linalg.norm(b) ** 2)
    return e_mse + regularization_term  # error function

In [ ]:
# MLP Optimization
err_out = np.zeros(len(Ns)*len(rhos))
err_test = np.zeros(len(Ns)*len(rhos)) 
Weights = np.zeros((len(Ns)*len(rhos), Ns[-1]*(Ns[-1]+n-1)))
rho_val = np.zeros(len(Ns)*len(rhos))
N_val = np.zeros(len(Ns)*len(rhos))
mse_test = np.zeros(len(Ns)*len(rhos))
mse_train = np.zeros(len(Ns)*len(rhos))
variance = np.zeros(len(Ns)*len(rhos))
cost_time = np.zeros(len(Ns)*len(rhos))
k = 0

for N in Ns:
    for rho in rhos:
        np.random.seed(2108602) 
        W0 = np.random.random(N*(N+n-1))
        Optimization_method = "L-BFGS-B"
        start = time.time()
        res = minimize(err_fn, W0, args=(train_data), method=Optimization_method, tol=1e-7)
        cost_time[k] = time.time() - start
        err_out[k] = round(res.fun, 3)
        Weights[k, 0:N*(N+n-1)] = res.x
        W = res.x
        w = np.reshape(W[0:n*N],(N,n))
        v = np.reshape(W[n*N:n*N+N],(N,1)) 
        b = np.reshape(W[n*N+N:n*N+2*N],(N,1))
        y_pred_test = y_hat(w, b, v, x_test)
        mse_test[k] = round(mse(y_pred_test, y_test), 3) 
        y_pred_train = y_hat(w, b, v, x_train)
        mse_train[k] = round(mse(y_pred_train, y_train), 3)
        variance[k] = round(mse_test[k] - mse_train[k], 3) 
        rho_val[k] = rho
        N_val[k] = N 
        print(f"Nodes: {N}, rho: {rho}, Regularized_Error: {err_out[k]}, MSE_Test: {mse_test[k]}, MSE_Train: {mse_train[k]}, Variance: {variance[k]}, Time: {cost_time[k]}") 
        k = k+1

In [ ]:
output_df = {
    'Rho': rho_val,
    'N_Neurons': N_val,
    'MSE_Test': mse_test,
    'MSE_Train': mse_train,
    'Variance': variance,
    'Regularized_Error': err_out,
    'Time': cost_time
}

In [ ]:
out_df = pd.DataFrame(output_df)
out_df.head()

In [ ]:

directory = 'F:\Sapienza Courses\optimization methods for machine learning\Project Assignment_1.1\Jupyter_1.1' 
file_path = directory + '\output_data.xlsx' 
out_df.to_excel(file_path, index=False) 


In [ ]:
"""
def plot_error(rhos, a, N):
    plt.title(f"Errors and rhos for {N} eurons")
    plt.xlabel("rhos")
    plt.ylabel("Errors")
    plt.xlim(0, max(rhos))
    plt.ylim(0, max(err_out[a:a+4]))
    plt.plot(rhos, err_out[a:a+4])
    plt.show() 
"""

In [ ]:
"""
a = 0
for N in Ns:
    plot_error(rhos, a, N)
    a+=4
""" 

In [ ]:
"""from matplotlib import projections
from mpl_toolkits.mplot3d import Axes3D
x1_vals = np.linspace(-2, 2, 50)
x2_vals = np.linspace(-3, 3, 50)
x1_mesh, x2_mesh = np.meshgrid(x1_vals, x2_vals)
x_mesh = np.column_stack((x1_mesh.flatten(), x2_mesh.flatten(), np.ones(len(x1_mesh.flatten()))))      
y = y_hat(w, b, v, x_mesh)
y_mesh = y.reshape(x1_mesh.shape)
fig = plt.figure(dpi = 150)
ax = plt.gca(projection = '3d')
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Predicted Values')
ax.plot_surface(x1_mesh, x2_mesh, y_mesh, cmap='viridis')"""
